In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color_pal = sns.color_palette()
import statsmodels.formula.api as smf
from scipy import stats

In [2]:
def download_link(url):
  '''Method reformates the share link from Google Drive into fetchable form'''
  return 'https://drive.google.com/uc?id=' + url.split('/')[-2]

In [3]:
weather_data = pd.read_csv(download_link("https://drive.google.com/file/d/1keTewFIu3ceNYhfNuiLxD9vqlcW5t2S9/view?usp=sharing")).fillna(0)
cycling_data = pd.read_csv(download_link("https://drive.google.com/file/d/15eHai6zkPwOBMq59n8uIjjohuuiaV8DF/view?usp=sharing"))

In [4]:
river_park_data = cycling_data[cycling_data['NAZOV'] == '#3 - River Park'].copy()
river_park_data['timestamp'] = pd.to_datetime(river_park_data['DATUM_A_CAS'])
river_park_data['total_count'] = river_park_data['POCET_DO'] + river_park_data['POCET_Z']
river_park_data['date'] = river_park_data['timestamp'].dt.date
river_park_daily = river_park_data.groupby('date')['total_count'].sum().reset_index().rename(columns={'total_count': 'count'})

In [5]:
vajanskeho_data = cycling_data[cycling_data['NAZOV'].isin(['#13 - Vajanského 1', '#14 - Vajanského 2'])].copy()
vajanskeho_data['timestamp'] = pd.to_datetime(vajanskeho_data['DATUM_A_CAS'])
vajanskeho_data['total_count'] = vajanskeho_data['POCET_DO'] + vajanskeho_data['POCET_Z']

# Pivot to separate columns for each sensor
vaj_wide = vajanskeho_data.pivot_table(
    index='timestamp',
    columns='NAZOV',
    values='total_count',
    aggfunc='sum'
).reset_index()

vaj_wide['Vajanského'] = vaj_wide['#13 - Vajanského 1'] + vaj_wide['#14 - Vajanského 2']

# Aggregate to daily totals
vaj_wide['date'] = vaj_wide['timestamp'].dt.date
vaj_daily = vaj_wide.groupby('date')['Vajanského'].sum().reset_index()

In [6]:
vaj_counter_start = vaj_daily['date'].min()
print(f"Vajanského counter data starts: {vaj_counter_start}")

Vajanského counter data starts: 2024-01-25


In [7]:
river_park_daily_pre = river_park_daily[river_park_daily['date'] < vaj_counter_start][['date','count']].copy()
vaj_daily_post = vaj_daily[vaj_daily['date'] >= vaj_counter_start].rename(columns={'Vajanského':'count'}).copy()

daily_data = pd.concat([river_park_daily_pre, vaj_daily_post], ignore_index=True)
daily_data['date'] = pd.to_datetime(daily_data['date'])

In [8]:
cycle_path_opening = pd.to_datetime('2023-09-01').date()

daily_data['post_opening'] = (daily_data['date'].dt.date >= cycle_path_opening).astype(int)

In [ ]:
weather_data_merge = weather_data[['date','tavg','prcp','wspd']].copy()
weather_data_merge['date'] = pd.to_datetime(weather_data_merge['date']).dt.date

daily_data['date_for_merge'] = daily_data['date'].dt.date

daily_data = daily_data.merge(
    weather_data_merge,
    left_on='date_for_merge',
    right_on='date',
    how='left'
).drop(['date_for_merge', 'date_y'], axis=1, errors='ignore')

daily_data = daily_data.rename(columns={'date_x': 'date', 'tavg':'temperature','prcp':'precipitation','wspd':'wind_speed'})
daily_data['date'] = pd.to_datetime(daily_data['date'])

In [10]:
# Add regression features
daily_data['is_weekend'] = daily_data['date'].dt.weekday.isin([5,6]).astype(int)
daily_data['days_since_start'] = (daily_data['date'] - daily_data['date'].min()).dt.days
daily_data['days_after_opening'] = daily_data['days_since_start'] - (cycle_path_opening - daily_data['date'].min().date()).days
daily_data['days_after_opening'] = daily_data['days_after_opening'].clip(lower=0)

In [11]:
model = smf.ols(
    'count ~ days_since_start + post_opening + days_after_opening + temperature + precipitation + wind_speed + is_weekend + C(date.dt.month)',
    data=daily_data.dropna()
)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 7})
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  count   R-squared:                       0.658
Model:                            OLS   Adj. R-squared:                  0.653
Method:                 Least Squares   F-statistic:                     99.12
Date:                Mon, 05 Jan 2026   Prob (F-statistic):          3.48e-221
Time:                        21:48:12   Log-Likelihood:                -8689.3
No. Observations:                1203   AIC:                         1.742e+04
Df Residuals:                    1184   BIC:                         1.751e+04
Df Model:                          18                                         
Covariance Type:                  HAC                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                616

In [12]:
def prepare_sensor_data(cycling_data, weather_data, sensor_names, sensor_column, cycle_path_opening='2023-09-01'):
    """
    Prepare sensor data for regression analysis.

    """    
    sensor_data = cycling_data[cycling_data['NAZOV'].isin(sensor_names)].copy()
    sensor_data['timestamp'] = pd.to_datetime(sensor_data['DATUM_A_CAS'])
    sensor_data['total_count'] = sensor_data['POCET_DO'] + sensor_data['POCET_Z']
    
    sensor_wide = sensor_data.pivot_table(
        index='timestamp',
        columns='NAZOV',
        values='total_count',
        aggfunc='sum'
    ).reset_index()
    
    if len(sensor_names) > 1:
        sensor_wide[sensor_column] = sensor_wide[sensor_names].sum(axis=1)
    else:
        sensor_wide[sensor_column] = sensor_wide[sensor_names[0]]
    
    sensor_wide['date'] = sensor_wide['timestamp'].dt.date
    daily_data = sensor_wide.groupby('date')[sensor_column].sum().reset_index()
    daily_data = daily_data.rename(columns={sensor_column: 'count'}).copy()
    daily_data['date'] = pd.to_datetime(daily_data['date'])
    
    cycle_path_opening = pd.to_datetime(cycle_path_opening).date()
    daily_data['post_opening'] = (daily_data['date'].dt.date >= cycle_path_opening).astype(int)
    
    weather_data_merge = weather_data[['date', 'tavg', 'prcp', 'wspd']].copy()
    weather_data_merge['date'] = pd.to_datetime(weather_data_merge['date']).dt.date
    
    daily_data['date_for_merge'] = daily_data['date'].dt.date
    
    daily_data = daily_data.merge(
        weather_data_merge,
        left_on='date_for_merge',
        right_on='date',
        how='left'
    ).drop(['date_for_merge', 'date_y'], axis=1, errors='ignore')
    
    daily_data = daily_data.rename(columns={
        'date_x': 'date',
        'tavg': 'temperature',
        'prcp': 'precipitation',
        'wspd': 'wind_speed'
    })
    daily_data['date'] = pd.to_datetime(daily_data['date'])
    
    daily_data['is_weekend'] = daily_data['date'].dt.weekday.isin([5, 6]).astype(int)
    daily_data['days_since_start'] = (daily_data['date'] - daily_data['date'].min()).dt.days
    daily_data['days_after_opening'] = daily_data['days_since_start'] - (
        cycle_path_opening - daily_data['date'].min().date()
    ).days
    daily_data['days_after_opening'] = daily_data['days_after_opening'].clip(lower=0)
    daily_data['month'] = daily_data['date'].dt.month
    
    return daily_data


def build_regression_model(data, formula, cov_type='HAC', maxlags=7):
    """
    Build OLS regression model with specified formula and covariance type.
    """
    
    clean_data = data.dropna()
    
    model = smf.ols(formula, data=clean_data)
    results = model.fit(cov_type=cov_type, cov_kwds={'maxlags': maxlags})
    
    return results, results.summary()


def model_pipeline(cycling_data, weather_data, sensor_config, formula):
    """
    Complete pipeline to prepare data and build regression model.
    """
    prepared_data = prepare_sensor_data(
        cycling_data,
        weather_data,
        sensor_names=sensor_config['sensor_names'],
        sensor_column=sensor_config['sensor_column'],
        cycle_path_opening=sensor_config.get('cycle_path_opening', '2023-09-01')
    )
    
    results, summary = build_regression_model(prepared_data, formula)
    
    return {
        'data': prepared_data,
        'results': results,
        'summary': summary,
        'formula': formula
    }


In [13]:
parickova_config = {
    'sensor_names': ['#9 - Páričkova'],
    'sensor_column': 'Parickova_count',
    'cycle_path_opening': '2023-09-01'
}

parickova_formula = 'count ~ temperature + precipitation + wind_speed + is_weekend + C(month)'

parickova_result = model_pipeline(cycling_data, weather_data, parickova_config, parickova_formula)

print(parickova_result['summary'])


                            OLS Regression Results                            
Dep. Variable:                  count   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     181.6
Date:                Mon, 05 Jan 2026   Prob (F-statistic):          3.65e-275
Time:                        21:48:12   Log-Likelihood:                -6682.9
No. Observations:                1035   AIC:                         1.340e+04
Df Residuals:                    1019   BIC:                         1.348e+04
Df Model:                          15                                         
Covariance Type:                  HAC                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        340.7311     22.214     15.

In [14]:
viedenska_config = {
    'sensor_names': ['Viedenska'],
    'sensor_column': 'Viedenska_count',
    'cycle_path_opening': '2023-09-01'
}

viedenska_formula = 'count ~ temperature + precipitation + wind_speed + is_weekend + C(month)'

viedenska_result = model_pipeline(cycling_data, weather_data, viedenska_config, viedenska_formula)

print(viedenska_result['summary'])


                            OLS Regression Results                            
Dep. Variable:                  count   R-squared:                       0.690
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     165.2
Date:                Mon, 05 Jan 2026   Prob (F-statistic):          2.70e-298
Time:                        21:48:13   Log-Likelihood:                -10001.
No. Observations:                1438   AIC:                         2.003e+04
Df Residuals:                    1422   BIC:                         2.012e+04
Df Model:                          15                                         
Covariance Type:                  HAC                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        236.3396     21.850     10.